<a href="https://colab.research.google.com/github/PlaZMaD/climate/blob/main/optuna_and_gaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install optuna

In [ ]:
from catboost.metrics import RMSE
from sklearn.model_selection import train_test_split
import optuna

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pylab as plt
import plotly.express as px
import plotly.graph_objects as go
# from google.colab import drive
import catboost
import xgboost
from sklearn.preprocessing import FunctionTransformer
import calendar
import json
# drive.mount('/content/drive')


In [ ]:
# !ls /content/drive/MyDrive/weather_data

In [ ]:
!gdown  1s5c7x3L6STewm7GBApVAEcqbYbTu45RO
# !gdown 1NT0GrfbUPdB8psXPS1k-wiVwc6A3oxgl

In [ ]:
fileName = 'для поиска предикторов.xlsx'
time = 'Date Time'
target = 'NEE'
targets = ['NEE', 'RECO', 'GPP', 'NEE_cor']
use_sl = False

In [ ]:
training_variables = ['Ta', 'RH', 'Pa', 'VPD', 'P', 'SWin', 'SWout', 'LWin', 'LWout', 'Rn', 'WTD', 'Ts', 'SHF']

In [ ]:
data_all = pd.read_excel(fileName, skiprows=lambda x: x==1, sheet_name=None)

In [ ]:
print(data_all.keys())
print(data_all['FYB15'].columns)

In [ ]:
# print(data.head())
# print(data.columns)
data_fyb = pd.concat([data_all[item] for item in ['FYB15', 'FYB16', 'FYB17', 'FYB18', 'FYB19', 'FYB20', 'FYB21', 'FYB22']], ignore_index=True)
data_upo = pd.concat([data_all[item] for item in ['UPO12', 'UPO17']], ignore_index=True)
data_muh = data_all['MUH']
data_plt = data_all['PLT']

In [ ]:
data_dict = {}
data_dict['MUH'] = data_muh
data_dict['PLT'] = data_plt
data_dict['UPO'] = data_upo
data_dict['FYB'] = data_fyb

In [ ]:
if False:
  for _, item in data_dict.items():
    item.index = item[time]
    item.resample('1D').mean()

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1# if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
      cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
      cols.append(df.shift(-i))
    # put it all together
    agg = pd.concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
      agg.dropna(inplace=True)
    return agg.values

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def add_time_features(data_f, time_col, year_sincos=True, hour_sincos=True):
  data = data_f.copy()
  data['day_of_year_f'] = data[time_col].dt.dayofyear
  data['year_f'] = data[time_col].dt.year
  years_in_dataset = data[time_col].dt.year.unique()
  n_Days_in_year = {i:366 if calendar.isleap(i) else 365 for i in years_in_dataset}
  new_features = []
  if year_sincos:
    new_features.append('year_sin')
    new_features.append('year_cos')
    for year in years_in_dataset:
      data.loc[data['year_f']==year, "year_sin"] = sin_transformer(n_Days_in_year[year]).fit_transform(data["day_of_year_f"])
      data.loc[data['year_f']==year, "year_cos"] = cos_transformer(n_Days_in_year[year]).fit_transform(data["day_of_year_f"])
  if hour_sincos:
    new_features.append('hour_sin')
    new_features.append('hour_cos')
    data['hour_sin'] = sin_transformer(24).fit_transform(data[time_col].dt.hour)
    data['hour_cos'] = sin_transformer(24).fit_transform(data[time_col].dt.hour)

  return data, new_features

def data_preprocessing(data, time_col):
  pass


def get_best_params(data_f, target, features, time_col, sl=False, sl_length=6, name=""):
  data, new_features = add_time_features(data_f, time_col)
  features = features + new_features
  print('features: ', features)
  data['process'] = np.invert(pd.isnull(data[target]))
  if sl:
    tmp_data = series_to_supervised(data[target], n_in=sl_length, dropnan=False)
    new_cols = [f'target_{i}' for i in range(tmp_data.shape[1])]
    new_sup_data = pd.DataFrame(tmp_data, columns=new_cols)
    data = pd.concat([data, new_sup_data])
    exist_cols = []
    for col in  new_cols:
      data[f'exists_{col}'] = np.invert(pd.isnull(data[col]))
      exist_cols.append(f'exists_{col}')
    features = features + new_cols

  training_data = data.query('process==True').copy()
  target_data = training_data[target].copy()
  training_data[features].fillna(method='ffill', inplace=True)
  training_data[features].fillna(0, inplace=True)

  X_train, X_test, y_train, y_test  = train_test_split(training_data[features], training_data[target])

  def objective(trial):
    # define the grid

    param = {
        "objective": "RMSE",
        "iterations":trial.suggest_int("iterations", 100, 1500),
        "learning_rate":trial.suggest_float('learning_rate',1e-3, 1.0, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 1, log=True),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "6gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    bst = catboost.CatBoostRegressor(**param)
    bst.fit(X_train, y_train)
    preds = bst.predict(X_test)
    pred_labels = np.rint(preds)
    # objective should return the metrics that you want to optimize
    accuracy = sklearn.metrics.mean_absolute_error(y_test, pred_labels)
    return accuracy

  study = optuna.create_study(direction="minimize")
  # you may increase the nubmer of trials in case you have enough time
  study.optimize(objective, n_trials=10, timeout=600)

  print("  Best value: {}".format(study.best_trial.value))
  print(study.best_params)
  return study.best_params
  # scaler = sklearn.preprocessing.StandardScaler()
  # clf_X = scaler.fit_transform(training_data[features])
  # clf_X = pd.DataFrame(clf_X, columns=features)
  # clf_y = target_data
  scores = sklearn.model_selection.cross_val_score(clf, clf_X, clf_y, cv=5, scoring='neg_mean_absolute_percentage_error')
  print(scores, scores.mean())
  clf_test = catboost.CatBoostRegressor(iterations=300,
                                  learning_rate=0.03,
                                  depth=10,
                                  l2_leaf_reg=1,
                                  grow_policy = 'Depthwise',
                                  loss_function=RMSE())
  clf_test.fit(clf_X, clf_y)

  print(clf_test.feature_importances_)

  feature_importance = clf_test.feature_importances_
  sorted_idx = np.argsort(feature_importance)
  fig = plt.figure(figsize=(12, 6))
  plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')

  if use_sl:
    plt.yticks(range(len(sorted_idx)), np.array(features)[sorted_idx])
  else:
    plt.yticks(range(len(sorted_idx)), np.array(features)[sorted_idx])
  plt.title(f'Feature Importance for {name}')
  plt.savefig(f"{name}.png")
  return {np.array(features)[i]:feature_importance[i] for i in sorted_idx}

In [ ]:
best_params = get_best_params(data_dict['FYB'], target, training_variables,  time)

In [ ]:
data = data_dict['FYB']
features = training_variables
data, new_features = add_time_features(data, time)
features = features + new_features

data['process'] = np.invert(pd.isnull(data[target]))
training_data = data.query('process==True').copy()

target_data = training_data[target].copy()

training_data[features] = training_data[features].fillna(method='ffill', inplace=False)
training_data[features].fillna(0, inplace=True)

X_train, X_test, y_train, y_test  = train_test_split(training_data[features], training_data[target])

In [ ]:
clf_test = catboost.CatBoostRegressor(**best_params)
clf_test.fit(X_train, y_train)

In [ ]:
my_predictions = clf_test.predict(X_test)
print(sklearn.metrics.mean_squared_error(y_test, my_predictions))
print(y_test.mean(), y_test.std())
plt.hist(y_test-my_predictions, bins=50, range=(-5, 5))
plt.show()

In [ ]:
# plot_df = pd.DataFrame({'y':y_test, 'error':y_test - my_predictions})

# plot_df['bins'] = pd.cut(plot_df['y'], 50)

# gp_data = plot_df.groupby('bins').mean()
# plt.plot(gp_data['y'], gp_data['error'])

# # plt.plot()
# plt.show()

In [ ]:
print(clf_test.feature_importances_)

feature_importance = clf_test.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
if use_sl:
  plt.yticks(range(len(sorted_idx)), np.array(training_variables+exist_cols)[sorted_idx])
else:
  plt.yticks(range(len(sorted_idx)), np.array(features)[sorted_idx])
plt.title('Feature Importance')


In [ ]:
data = data_dict['MUH']
features = training_variables
data, new_features = add_time_features(data, time)
features = features + new_features

data['process'] = np.invert(pd.isnull(data[target]))
training_data = data.query('process==True').copy()

target_data = training_data[target].copy()

training_data[features] = training_data[features].fillna(method='ffill', inplace=False)
training_data[features].fillna(0, inplace=True)

clf_test = catboost.CatBoostRegressor(**best_params)
clf_test.fit(training_data[features], training_data[target])


In [ ]:
data['prediction'] = clf_test.predict(data[features])
data['plot'] = data[target]
data.loc[data['process']==False, 'plot'] = data['prediction']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data[time], y=data[target]))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.loc[data['process']==True, time], y=data.loc[data['process']==True, 'plot'],mode='markers', name='real'))
# fig.add_trace(go.Scatter(x=data[time], y=data['plot'], mode='markers', marker=dict(color=data['process'].astype('int'))))
fig.add_trace(go.Scatter(x=data.loc[data['process']==False, time], y=data.loc[data['process']==False, 'plot'], mode='markers',name='filled'))
fig.show()